## imports:

In [9]:
from utils.geometry import forward_backward_consistency_check
import os
import numpy as np
import torch
from torchvision.transforms import v2

## Paths:

In [10]:
sources = "../sources"
images_dir = sources+"/images"
flow_res = sources+"/flows"
m_dir = flow_res + "/M"
flow_res_fwd = flow_res + "/fwd"
flow_res_bwd = flow_res + "/bwd"
mod = images_dir + "/demo_mod"
ref = images_dir + "/demo_ref"
Xm1 = mod + "/openimage_0995.png"
Xo1 = ref + "/openimage_0995.png"
res_example = images_dir + "/res_example"
flownet = flow_res + "/FlowNet2.flo"
spynet = flow_res + "/openimage_0995.flo"
openimage_0995_res = res_example + "/openimage_0995_ref_like.png"
checkpoints = "../../checkpoints"
flownet2_checkpoint = checkpoints + "/FlowNet2/FlowNet2_checkpoint.pth.tar"

## create optical flow via SpyNet:

In [11]:
!python modules/SpyNet/pred.py --model sintel-final --one $mod --two $ref --out $flow_res_fwd

/home/din/miniconda3/envs/buna/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608935911/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


In [12]:
!python modules/SpyNet/pred.py --model sintel-final --one $ref --two $mod --out $flow_res_bwd

/home/din/miniconda3/envs/buna/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608935911/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


## create optical flow via FlowNet2:

In [13]:
#add checkpoints download in the readme and run FlowNet2/main.py

In [14]:
print(flow_res_bwd,flow_res_fwd)


../sources/flows/bwd ../sources/flows/fwd


## Flow Consistency Mask

In [15]:
from torchvision.transforms import v2
G = v2.GaussianBlur(kernel_size=(7, 7), sigma=(0.1, 5.))


class SeedSetter:
    def __init__(self, seed):
        self.seed = seed

    def __enter__(self):
        self.numpy_state = np.random.get_state()
        self.torch_state = torch.get_rng_state()

        np.random.seed(self.seed)
        torch.manual_seed(self.seed)

    def __exit__(self, exc_type, exc_val, exc_tb):
        np.random.set_state(self.numpy_state)
        torch.set_rng_state(self.torch_state)

# Usage with a 'with' statement
seed_value = 42

In [19]:
from utils.flow_utils import readFlow
from utils.tools import toTensor, grouped_files_from_dir_list
import torch
from losses import L1

path_lst = grouped_files_from_dir_list([flow_res_fwd,flow_res_bwd])
for pair in path_lst:
    fwd_name , bwd_name = pair
    fwd = toTensor(readFlow(fwd_name))
    bwd = toTensor(readFlow(bwd_name))
    M_inconsistent , _ = forward_backward_consistency_check(fwd,bwd,alpha=0.85,beta=0.085)
    with SeedSetter(seed_value):
        M = 1 - G(M_inconsistent)
    #TO DO: save M in source dir
    name_img = fwd_name[:-4]
    name_img = name_img.split('/')[-1]
    flow_path = m_dir + "/" + name_img + ".pt"
    torch.save(M, flow_path)
#    tmp_M=torch.load(flow_path)
#    diff = tmp_M-M
#    print(torch.sum(diff))

    

/home/din/miniconda3/envs/buna/lib/python3.10/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1712608935911/work/aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [22]:

l1 = L1()
path_lst = grouped_files_from_dir_list([flow_res_fwd,m_dir])
for pair in path_lst:
    flo , m = pair
    flo = toTensor(readFlow(flo))
    m = torch.load(m).unsqueeze(0)
    print(l1(m,0))


tensor(0.9674, device='cuda:0')
tensor(0.9481, device='cuda:0')


In [ ]:
"""
tensor(0.9674, device='cuda:0')
tensor(0.9481, device='cuda:0')
"""